**Exploración con IA Generativa – LangChain**



Este notebook explora el uso de **IA Generativa (LLMs)** para complementar
el pipeline de Machine Learning del proyecto MetLife.

Objetivos:
- Generar explicaciones en lenguaje natural de predicciones
- Analizar perfiles de clientes
- Simular escenarios “what-if”


In [1]:
#=======================================
#Importar librerias
#========================================
import pandas as pd
import numpy as np
import joblib

from transformers import pipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_community.llms import HuggingFacePipeline




c:\Users\zukan\Desktop\metlife_ml\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\zukan\Desktop\metlife_ml\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
#====================================
#Carga del dataset
#====================================

df = pd.read_csv("../data/dataset.csv")
df.head()



,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
#==============================================
#Cargamos el modelo real entrenado
#==============================================

model = joblib.load("../models/best_model.joblib")
print("Modelo cargado correctamente:", type(model))


Modelo cargado correctamente: <class 'sklearn.pipeline.Pipeline'>


In [4]:
#=====================================
#Seleccionamos un cliente
#=====================================
client = df.sample(1, random_state=42).iloc[0]
client


age                 45
sex             female
bmi             25.175
children             2
smoker              no
region       northeast
charges     9095.06825
Name: 764, dtype: object

In [5]:
#====================================
#Preparar input y predecir con el modelo real
#=====================================

X_client = pd.DataFrame([client.drop("charges")])

predicted_cost = model.predict(X_client)[0]
predicted_cost


np.float64(10359.789485747318)

In [6]:
#==============================
#Inicializar LLM open-source
#==============================

hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    framework="pt",        
    max_length=300
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)



Device set to use cpu
C:\Users\zukan\AppData\Local\Temp\ipykernel_6708\269448780.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [11]:
from langchain_core.prompts import PromptTemplate


# ==============================
# Crear cadena LangChain (nuevo estilo)
# ==============================

prompt = PromptTemplate(
    input_variables=["input"],
    template="""
Sos un analista de datos senior en seguros de salud.

Dado el siguiente input:
{input}

Explicá en lenguaje claro:
- Qué factores influyen en el costo médico
- Por qué el modelo puede predecir un valor alto o bajo
- Qué insights de negocio se pueden extraer
"""
)

chain = prompt | llm


In [13]:

# ===============================
# Preparar input para IA generativa
# ===============================

input_text = f"""
Cliente con las siguientes características:
- Edad: {int(client['age'])}
- BMI: {float(client['bmi'])}
- Fumador: {client['smoker']}
- Hijos: {int(client['children'])}
- Región: {client['region']}

El modelo de Machine Learning predijo un costo médico anual de: {round(predicted_cost, 2)} USD.
"""

# ===============================
# Generar explicación con IA Generativa
# ===============================

response = chain.invoke({"input": input_text})

print(response)



A client with the following characteristics: - Age: 45 - BMI: 25.175 - Fumador: no - Hijos: 2 - Región: northeast El modelo de Machine Learning predijo un costo médico anual de: 10359.79 USD. Explicá en lenguaje claro: - What factors influyen en el costo médico - Por qué el modelo puede predecir un valor alto o bajo - A client with the following characteristics: - Age: 45 - BMI: 25.175 - Fumador: no - Hijos: 2 - Región: northeast El modelo de Machine Learning predijo un costo médico anual de: 10359.79 USD. Explicá en lenguaje claro: - What factors influyen en el costo médico - Por qué el modelo puede predecir un valor alto o bajo - A client with the following characteristics: - Age: 45 - BMI: 25.175 - Fumador: no - Hijos: 2 - Región: northeast El modelo de Machine Learning predijo un


In [14]:
#========================================
#Caso de escenario
#========================================


def scenario_analysis(client, model):
    scenarios = [
        ("No fumador", "no"),
        ("Fumador", "yes")
    ]
    
    results = []
    for label, smoker in scenarios:
        modified = client.copy()
        modified["smoker"] = smoker
        
        X_mod = pd.DataFrame([modified.drop("charges")])
        cost = model.predict(X_mod)[0]
        
        results.append((label, round(cost, 2)))
    
    return results

scenario_analysis(client, model)


[('No fumador', np.float64(10359.79)), ('Fumador', np.float64(25386.94))]

## Conclusion


Este notebook demuestra cómo se puede utilizar la IA Generativa para mejorar un flujo de trabajo de aprendizaje automático tradicional, proporcionando interpretabilidad y análisis de escenarios.

Si bien el modelo predictivo sigue siendo el aprendizaje automático clásico, los LLM aportan valor al acortar la distancia entre las predicciones técnicas y la comprensión del negocio.
